In [2]:
import urllib
from dotenv import load_dotenv
import os
import IPython
import json
import pandas as pd
import numpy as np
import openai
import comet_llm

In [3]:
# function to generate final results from model after calling OpenAI API
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return response.choices[0].message.content

# Load Data

In [19]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


def print_markdown(text):
    """Prints  text as markdown"""
    IPython.display.display(IPython.display.Markdown(text))


# load validation data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/article-tags.json")
val_data = json.load(f)


# load few shot data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/few_shot.json")
few_shot_data = json.load(f)

In [5]:
# helper functions to obtain final predictions from the model given a prompt template (e.g. zero-shot or few-shot) and the provided input data

def get_predictions(prompt_template, inputs):
    responses = []

    for i in range(len(inputs)):
        messages = [
            {
                "role": "system",
                "content": prompt_template.format(input=inputs[i])
            }
        ]

        response = get_completion(messages)
        responses.append(response)
        
    return responses

# Few-Shot Template

In [6]:
# function to define the few-shot template
def get_few_shot_template(few_shot_prefix, few_shot_suffix, few_shot_examples):
    return few_shot_prefix + "\n" +"\n".join([ "Abstract: "+ ex["abstract"] + "\n" + "Tags: " + str(ex["tags"]) + "\n" for ex in few_shot_examples]) + "\n\n" + few_shot_suffix

# function to sample few shot data
def random_sample_data (data, n):
    return np.random.choice(few_shot_data, n, replace=False)


# the few-shot prefix and suffix
few_shot_prefix = """Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]
Use the following examples to help with steering your responses:
"""
few_shot_suffix = """Abstract: {input}\nTags:"""

# load 3 samples from few shot data
few_shot_template = get_few_shot_template(few_shot_prefix, few_shot_suffix, random_sample_data(few_shot_data, 3))

In [7]:
few_shot_template

'Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format ["model_name"]. If you don\'t find model names in the abstract or you are not sure, return ["NA"]\nUse the following examples to help with steering your responses:\nAbstract: Children\'s drawings have a wonderful inventiveness, creativity, and variety to them. We present a system that automatically animates children\'s drawings of the human figure, is robust to the variance inherent in these depictions, and is simple and straightforward enough for anyone to use. We demonstrate the value and broad appeal of our approach by building and releasing the Animated Drawings Demo, a freely available public website that has been used by millions of people around the world. We present a set of experiments exploring the amount of training data needed for fine-tuning, as well as a perceptual study demonstrating the appeal of a novel twisted perspective retargetin

# Zero-Shot Template

In [8]:
zero_shot_template = """
Your task is extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]

Abstract: {input}
Tags:
"""

# Get Prediction

In [9]:
# using the validation data as inputs

abstracts = [val_data[i]["abstract"] for i in range(len(val_data))]
few_shot_predictions = get_predictions(few_shot_template, abstracts)
zero_shot_predictions = get_predictions(zero_shot_template, abstracts)
expected_tags = [str(val_data[i]["tags"]) for i in range(len(val_data))]

In [13]:
print("Few shot predictions")
print(few_shot_predictions)
print("\nZero shot predictions")
print(zero_shot_predictions)
print("\nExpected tags")
print(expected_tags)

Few shot predictions
["['Evol-Instruct', 'LLaMA', 'WizardLM', 'ChatGPT']", "['FLAN-T5', 'AMR', 'AMR2.0', 'AMR3.0', 'BioAMR', 'LoRA']", "['NA']", "['PAXQA', 'QG']", "['ChatGPT']", "['ViT', 'OpenCLIP']", "['SAM', 'IA', 'AIGC', 'Stable Diffusion']", "['Anything-3D', 'BLIP', 'Segment-Anything', 'text-to-image diffusion model']", "['Chameleon', 'GPT-4', 'ChatGPT']", "['NA']"]

Zero shot predictions
['["Evol-Instruct", "LLaMA", "WizardLM", "OpenAI ChatGPT"]', '["FLAN-T5"]', '["large language models", "generative AI", "hypothesis machines"]', '["PAXQA"]', '["ChatGPT"]', '["ViT", "OpenCLIP"]', '["Segment-Anything Model (SAM)", "Inpaint Anything (IA)", "AIGC models", "Stable Diffusion", "Inpaint Anything (IA)"]', '["Anything-3D", "BLIP", "Segment-Anything", "text-to-image diffusion model", "neural radiance field"]', '["Chameleon", "GPT-4", "ScienceQA", "TabMWP", "ChatGPT"]', '["NA"]']

Expected tags
["['LLaMA', 'ChatGPT', 'WizardLM']", "['FLAN-T5', 'FLAN']", "['NA']", "['PAXQA']", "['ChatGPT']"

# Log Prompt Results
Log prompt & results to Comet.

Track both prompt few-shot and zero-shot with the metadata and tags.

In [18]:
COMET_API_KEY = os.getenv("COMET_API_KEY")
COMET_WORKSPACE = os.getenv("COMET_WORKSPACE")


# initialize comet
comet_llm.init(COMET_API_KEY, COMET_WORKSPACE, project="ml-paper-tagger-prompts")

# log the predictions
for i in range(len(expected_tags)):
    # log the few-shot predictions
    comet_llm.log_prompt(
        prompt=few_shot_template.format(input=abstracts[i]),
        prompt_template=few_shot_template,
        output=few_shot_predictions[i],
        tags = ["gpt-3.5-turbo", "few-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

    # log the zero-shot predictions
    comet_llm.log_prompt(
        prompt=zero_shot_template.format(input=abstracts[i]),
        prompt_template=zero_shot_template,
        output=zero_shot_predictions[i],
        tags = ["gpt-3.5-turbo", "zero-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

COMET INFO: Valid Comet API Key saved in C:\Users\achma\.comet.config (set COMET_CONFIG to change where it is saved).


Prompt logged to https://www.comet.com/llmops-exercise/ml-paper-tagger-prompts
